## Install depedncies

In [ ]:
!pip install -U langchain langchain-community langchain-experimental replicate pandas

## Import Library

In [ ]:
import os
import pandas as pd
from google.colab import userdata
from langchain_community.llms import Replicate
from langchain_experimental.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent
from tqdm import tqdm


## Set API Token for Replicate

In [ ]:
api_token = userdata.get('api_token')
os.environ["REPLICATE_API_TOKEN"] = api_token

## Load IBM granite

In [ ]:
llm = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct"
)

## Load Dataset

In [ ]:
def load_fasttext_file(filepath):
    data = []
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split(" ", 1)
            if len(parts) == 2:
                label, review = parts
                data.append((label, review))
    return pd.DataFrame(data, columns=["label", "review"])

df = load_fasttext_file("test.ft.txt")

label_map = {"__label__1": "negative", "__label__2": "positive"}
df["sentiment_label"] = df["label"].map(label_map)

print("✅ Dataset Loaded Successfully")
print(df.head(10))

✅ Dataset Loaded Successfully
        label                                             review  \
0  __label__2  Great CD: My lovely Pat has one of the GREAT v...   
1  __label__2  One of the best game music soundtracks - for a...   
2  __label__1  Batteries died within a year ...: I bought thi...   
3  __label__2  works fine, but Maha Energy is better: Check o...   
4  __label__2  Great for the non-audiophile: Reviewed quite a...   
5  __label__1  DVD Player crapped out after one year: I also ...   
6  __label__1  Incorrect Disc: I love the style of this, but ...   
7  __label__1  DVD menu select problems: I cannot scroll thro...   
8  __label__2  Unique Weird Orientalia from the 1930's: Exoti...   
9  __label__1  Not an "ultimate guide": Firstly,I enjoyed the...   

  sentiment_label  
0        positive  
1        positive  
2        negative  
3        positive  
4        positive  
5        negative  
6        negative  
7        negative  
8        positive  
9        negative  


## Promt for sentiment classification

## Sentiment Classification (row-by-row)

In [ ]:
def classify_review(text):
    prompt = f"""
    Classify the following product review into one of these categories:
    - positive
    - negative
    - neutral

    Review: {text}
    Answer with only one word: positive, negative, or neutral.
    """
    try:
        result = llm.invoke(prompt)
        return result.strip().lower()
    except Exception:
        return "error"

sample_df = df.head(50).copy()
preds = []
for r in tqdm(sample_df["review"]):
    preds.append(classify_review(r))
sample_df["predicted_sentiment"] = preds

100%|██████████| 50/50 [00:42<00:00,  1.17it/s]


## Analytical Result

In [ ]:
print("\n=== Analytical Result ===")
print(sample_df[["review", "sentiment_label", "predicted_sentiment"]].head(10))

distribution = sample_df["predicted_sentiment"].value_counts()
print("\nDistribution:")
print(distribution)

accuracy = (sample_df["predicted_sentiment"] == sample_df["sentiment_label"]).mean()
print(f"\nAccuracy: {accuracy:.2%}")


=== Analytical Result ===
                                              review sentiment_label  \
0  Great CD: My lovely Pat has one of the GREAT v...        positive   
1  One of the best game music soundtracks - for a...        positive   
2  Batteries died within a year ...: I bought thi...        negative   
3  works fine, but Maha Energy is better: Check o...        positive   
4  Great for the non-audiophile: Reviewed quite a...        positive   
5  DVD Player crapped out after one year: I also ...        negative   
6  Incorrect Disc: I love the style of this, but ...        negative   
7  DVD menu select problems: I cannot scroll thro...        negative   
8  Unique Weird Orientalia from the 1930's: Exoti...        positive   
9  Not an "ultimate guide": Firstly,I enjoyed the...        negative   

  predicted_sentiment  
0            positive  
1            positive  
2            negative  
3            negative  
4            positive  
5            negative  
6           

## Insight & Findings

In [ ]:
insight_prompt = f"""
You are a data analyst. Based on the following sentiment analysis results,
summarize the insights in clear and logical points.
Focus on what the distribution and accuracy imply about customer sentiment.

Distribution:
{distribution.to_dict()}
Accuracy: {accuracy:.2%}
"""
insights = llm.invoke(insight_prompt)
print("\n=== Insight & Findings ===")
print(insights)


=== Insight & Findings ===
1. The sentiment analysis results indicate a predominantly positive customer sentiment, with 27 instances of positive feedback compared to 22 instances of negative feedback and only 1 instance of neutral feedback. This suggests that customers generally have a favorable view of the product or service under examination.

2. The high accuracy rate of 96.00% implies that the sentiment analysis model is highly reliable and accurately identifies customer sentiments. This confidence in the model's performance allows for robust decision-making based on the sentiment analysis results.

3. The significant gap between positive and negative sentiments (27 vs. 22) indicates a strong positive bias among customers. This could imply that the product or service meets or exceeds customer expectations in most cases, leading to a positive overall experience.

4. The minimal neutral feedback (1 instance) suggests that customers are either strongly satisfied or dissatisfied, with

## Recommendations

In [ ]:
recommendation_prompt = f"""
Based on the insights below, provide 3-5 clear, concrete, and actionable
recommendations for an e-commerce company to improve their business.

Insights:
{insights}
"""
recommendations = llm.invoke(recommendation_prompt)
print("\n=== Recommendations ===")
print(recommendations)


=== Recommendations ===
Based on the insights provided, here are five actionable recommendations for the e-commerce company to improve their business:

1. **Capitalize on Positive Sentiment**: Given the strong positive bias, the company should leverage this by reinforcing successful aspects in marketing campaigns. Utilize positive customer testimonials and reviews in advertisements, social media, and email marketing to attract new customers and build trust.

2. **Investigate Negative Feedback**: Despite the overall positive sentiment, the 22 instances of negative feedback represent a significant opportunity for improvement. Conduct a thorough analysis of these negative reviews to identify recurring issues or patterns. Address these concerns by implementing product improvements, adjusting policies, or enhancing customer support processes.

3. **Personalize Customer Experiences**: With minimal neutral feedback, it's evident that customers either love or have issues with the product/serv